In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2, rasterio, json, itertools, math, os

from tqdm import tqdm
from IPython.display import SVG
from tensorflow.keras.utils import plot_model, model_to_dot, to_categorical, Sequence
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import *

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import (Add, Input, Conv2D, Dropout, Activation, BatchNormalization, MaxPooling2D, ZeroPadding2D, 
                                     AveragePooling2D, Flatten, Dense, Concatenate)
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import TensorBoard, Callback, LearningRateScheduler
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *

In [ ]:
def show_final_history(history):
    
    plt.style.use("ggplot")
    fig, ax = plt.subplots(1,2,figsize=(15,5))
    
    ax[0].set_title('Loss')
    ax[1].set_title('Accuracy')
    
    ax[0].plot(history.history['loss'], 'r-', label='Training Loss')
    ax[0].plot(history.history['val_loss'], 'g-', label='Validation Loss')
    ax[1].plot(history.history['accuracy'], 'r-', label='Training Accuracy')
    ax[1].plot(history.history['val_accuracy'], 'g-', label='Validation Accuracy')
    
    ax[0].legend(loc='upper right')
    ax[1].legend(loc='lower right')
    
    plt.show();
    pass

In [ ]:
def plot_learning_rate(loss_history):
    
    plt.style.use("ggplot")
    
    plt.plot(np.arange(0,len(loss_history.lr)), loss_history.lr)
    plt.show();
    pass

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f'
    thresh = cm.max()/2.0
    
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        
        plt.text(j,i, format(cm[i,j], fmt),
                horizontalalignment = 'center',
                color = "white" if cm[i,j] > thresh else "black")
        pass
    
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.grid(False);
    pass

In [ ]:
with open("../input/eurosat-dataset/EuroSATallBands/label_map.json","r") as f:
    class_names_encoded = json.load(f)
    pass

class_names = list(class_names_encoded.keys())
num_classes = len(class_names)
class_names_encoded

In [ ]:
bands = {'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'8a':9,'9':10,'10':11,'11':12,'12':13}

In [ ]:
def Normalise(arr_band):
    
    arr_band = Normalizer().fit_transform(arr_band)
    return arr_band

In [ ]:
train_df = pd.read_csv("../input/eurosat-dataset/EuroSATallBands/train.csv")
train_labels = train_df.loc[:,'Label']
train_labels = np.array(train_labels)

num_train_samples = train_labels.shape[0]

val_df = pd.read_csv("../input/eurosat-dataset/EuroSATallBands/validation.csv")
val_labels = val_df.loc[:,'Label']
val_labels = np.array(val_labels)

num_val_samples = val_labels.shape[0]

num_train_samples, num_val_samples

In [ ]:
basePath = "../input/eurosat-dataset/EuroSATallBands/"

def data_generator(csv_file, num_classes, band, batchSize = 10, targetSize = 64):
    
    df = pd.read_csv(csv_file)
    df.drop(columns=df.columns[0], inplace=True)
    num_samples = df.shape[0]
    
    while True:
        
        for offset in range(0, num_samples, batchSize):
            batchSamplesIdx = df.index[offset:offset+batchSize]
            
            X, y = [], []
            
            for i in batchSamplesIdx:
                img_name = df.loc[i,'Filename']
                label = df.loc[i,'Label']
                
                src = rasterio.open(os.path.join(basePath, img_name))
                
                arr_band = src.read(bands[band])
                arr_band = np.array(arr_band, dtype=np.float32)
                
                arr_band = Normalise(arr_band)
                
                X.append(arr_band)
                y.append(label)
                pass
            
            X = np.array(X)
            y = np.array(y)
            y = to_categorical(y, num_classes = num_classes)
            
            yield X, y
            pass
        pass
    pass

In [ ]:
def obtain_tif_images(csv_file, band):
    
    df = pd.read_csv(csv_file)
    df.drop(columns=df.columns[0], inplace=True)
    num_samples = df.shape[0]
    
    X, y = [], []
    
    for i in tqdm(range(num_samples)):
        
        label = df.loc[i,'Label']
        tif_name = df.loc[i,'Filename']
        tif_img = rasterio.open(os.path.join(basePath, tif_name))
        
        arr_band = np.array(tif_img.read(bands[band]), dtype=np.float32)
        arr_band = Normalise(arr_band)
        
        X.append(arr_band)
        y.append(label)
        
        pass
    
    X = np.array(X)
    y = np.array(y)
    
    return X,y
    pass


In [ ]:
def convBlock(X, filters, k, stage, block, s=1):
    
    conv_base_name = 'conv_' + str(stage) + block + '_branch'
    bn_base_name = 'bn_' + str(stage) + block + '_branch'
    ac_base_name = 'ac_' + str(stage) + block + '_branch'    
    
    F1 = filters
    
    X = Conv2D(filters=F1, kernel_size=(k,k), strides=(s,s), padding='same', name=conv_base_name+'a')(X)
    X = BatchNormalization(name=bn_base_name+'a')(X)
    X = Activation("relu", name=ac_base_name+'a')(X)
    
    return X
    pass

In [ ]:
def convModel(input_shape, numClasses):
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((5,5), name='zero_padding_1')(X_input)
    
    X = Conv2D(8, (3,3), strides=(1,1), padding='same', name="conv1")(X)
    X = BatchNormalization(name="bn_conv1")(X)
    
    X = convBlock(X, 16, 3, 2, block='A', s=1)
    X = MaxPooling2D((3,3), name="max_pooling_2")(X)
    X = Dropout(0.25, name="dropout_2")(X)
    
    X = convBlock(X, 16, 5, 3, block='A', s=1)
    X = MaxPooling2D((3,3), name="max_pooling_3")(X)
    X = Dropout(0.25, name="dropout_3")(X)
    
    X = Flatten(name="flatten_1")(X)
    
    X = Dense(32, name="dense_1")(X)
    X = Activation("relu", name="dense_relu_1")(X)
    X = Dropout(0.25, name="dense_dropout_1")(X)
    
    X = Dense(numClasses, activation="softmax", name="fc"+str(numClasses))(X)
    
    model = Model(inputs=X_input, outputs=X, name="Band_Class_Model")
    
    return model
    pass

# Band 2 [10m]

In [ ]:
band2_model = convModel(input_shape = (64,64,1), numClasses=10)

In [ ]:
plot_model(band2_model, to_file="band-class-model.png", show_shapes=True, show_layer_names=True)
SVG(model_to_dot(band2_model).create(prog="dot", format="svg"))

In [ ]:
band2_model.summary()

In [ ]:
band2_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", num_classes=10, batchSize=10, band='2')
band2_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", num_classes=10, batchSize=10, band='2')

In [ ]:
opt = Adam(lr=1e-5)
band2_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band2_history = band2_model.fit(band2_train_generator,
                                steps_per_epoch = num_train_samples//batchSize,
                                epochs = epochs,
                                verbose = 1,
                                validation_data = band2_val_generator,
                                validation_steps = num_val_samples//batchSize
                                )

In [ ]:
show_final_history(band2_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='2')

test_labels_encoded = to_categorical(test_labels, num_classes = 10)

test_tifs.shape, test_labels.shape, test_labels_encoded.shape

In [ ]:
test_pred2 = band2_model.predict(test_tifs)
test_pred2 = np.argmax(test_pred2, axis=1)
test_pred2.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred2)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 2 Confusion Matrix")
plt.grid(False);
plt.show();

# Band 3 [10m]

In [ ]:
band3_model = convModel(input_shape = (64,64,1), numClasses=10)

In [ ]:
band3_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", num_classes=10, batchSize=10, band='3')
band3_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", num_classes=10, batchSize=10, band='3')

In [ ]:
opt = Adam(lr=1e-4)
band3_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band3_history = band3_model.fit(band3_train_generator,
                                steps_per_epoch = num_train_samples//batchSize,
                                epochs = epochs,
                                verbose = 1,
                                validation_data = band3_val_generator,
                                validation_steps = num_val_samples//batchSize 
                                )

In [ ]:
show_final_history(band3_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='3')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred3 = band3_model.predict(test_tifs)
test_pred3 = np.argmax(test_pred3, axis=1)
test_pred3.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred3)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 3 Confusion Matrix")
plt.grid(False);
plt.show();

# Band 4 [10m]

In [ ]:
band4_model = convModel(input_shape = (64,64,1), numClasses=10)

In [ ]:
band4_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", num_classes=10, batchSize=10, band='4')
band4_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", num_classes=10, batchSize=10, band='4')

In [ ]:
opt = Adam(lr=1e-5)
band4_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band4_history = band4_model.fit(band4_train_generator,
                                steps_per_epoch = num_train_samples//batchSize,
                                epochs = epochs,
                                verbose = 1,
                                validation_data = band4_val_generator,
                                validation_steps = num_val_samples//batchSize 
                                )

In [ ]:
show_final_history(band4_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='4')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred4 = band4_model.predict(test_tifs)
test_pred4 = np.argmax(test_pred4, axis=1)
test_pred4.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred4)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 4 Confusion Matrix")
plt.grid(False);
plt.show();

# Band 5 [20m]

In [ ]:
band5_model = convModel(input_shape=(64,64,1), numClasses=10)

In [ ]:
band5_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='5', num_classes=10, batchSize=10)
band5_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", num_classes=10, batchSize=10, band='5')

In [ ]:
opt = Adam(lr=1e-5)
band5_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band5_history = band5_model.fit(band5_train_generator,
                                steps_per_epoch = num_train_samples//batchSize,
                                epochs = epochs,
                                verbose = 1,
                                validation_data = band5_val_generator,
                                validation_steps = num_val_samples//batchSize 
                                )

In [ ]:
show_final_history(band5_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='5')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred5 = band5_model.predict(test_tifs)
test_pred5 = np.argmax(test_pred5, axis=1)
test_pred5.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred5)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title = "Band 5 Confusion Matrix")
plt.grid(False)
plt.show();

# Band 6 [20m]

In [ ]:
band6_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='6', batchSize=10, num_classes=10)
band6_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", band='6', batchSize=10, num_classes=10)

In [ ]:
band6_model = convModel(input_shape=(64,64,1), numClasses=10)

In [ ]:
opt = Adam(lr=1e-5)
band6_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band6_history = band6_model.fit(band6_train_generator,
                               steps_per_epoch = num_train_samples//batchSize,
                               epochs = epochs,
                               validation_data = band6_val_generator,
                               validation_steps = num_val_samples//batchSize
                               )

In [ ]:
show_final_history(band6_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='6')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred6 = band6_model.predict(test_tifs)
test_pred6 = np.argmax(test_pred6, axis=1)
test_pred6.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred6)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 6 confusion matrix")
plt.grid(False)
plt.show()

# Band 7 [20m]

In [ ]:
band7_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='7', batchSize = 10, num_classes=10)
band7_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", band='7', batchSize=10, num_classes=10)

In [ ]:
band7_model = convModel(input_shape=(64,64,1), numClasses = 10)

In [ ]:
opt = Adam(lr=1e-5)
band7_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band7_history = band7_model.fit(band7_train_generator,
                               steps_per_epoch = num_train_samples//batchSize,
                               epochs = epochs,
                               validation_data = band7_val_generator,
                               validation_steps = num_val_samples//batchSize,
                               verbose = 1)

In [ ]:
show_final_history(band7_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='7')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred7 = band7_model.predict(test_tifs)
test_pred7 = np.argmax(test_pred7, axis=1)
test_pred7.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred7)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 7 confusion matrix")
plt.grid(False)
plt.show();

# Band 8 [10m]

In [ ]:
band8_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='8', batchSize = 10, num_classes = 10)
band8_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", band='8', batchSize=10, num_classes=10)

In [ ]:
band8_model = convModel(input_shape=(64,64,1), numClasses=10)

In [ ]:
opt = Adam(lr=1e-5)
band8_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band8_history = band8_model.fit(band8_train_generator,
                               steps_per_epoch = num_train_samples//batchSize,
                               epochs = epochs,
                               validation_data = band8_val_generator,
                               validation_steps = num_val_samples//batchSize,
                               verbose = 1)

In [ ]:
show_final_history(band8_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='8')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred8 = band8_model.predict(test_tifs)
test_pred8 = np.argmax(test_pred8, axis=1)
test_pred8.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred8)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 8 Confusion Matrix")
plt.grid(False)
plt.show()

# Band 8A [20m]

In [ ]:
band8a_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='8a', batchSize=10, num_classes=10)
band8a_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", band='8a', batchSize=10, num_classes=10)

In [ ]:
band8a_model = convModel(input_shape=(64,64,1), numClasses=10)

In [ ]:
opt = Adam(lr=1e-5)
band8a_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band8a_history = band8a_model.fit(band8a_train_generator,
                                 steps_per_epoch = num_train_samples//batchSize,
                                 epochs = epochs,
                                 validation_data = band8a_val_generator,
                                 validation_steps = num_val_samples//batchSize,
                                 verbose = 1)

In [ ]:
show_final_history(band8a_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='8a')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred8a = band8a_model.predict(test_tifs)
test_pred8a = np.argmax(test_pred8a, axis=1)
test_pred8a.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred8a)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 8A Confusion Matrix")
plt.grid(False)
plt.show()

# Band 11 [20m]

In [ ]:
band11_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='11', batchSize=10, num_classes=10)
band11_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", band='11', batchSize=10, num_classes=10)

In [ ]:
band11_model = convModel(input_shape=(64,64,1), numClasses=10)

In [ ]:
opt = Adam(lr=1e-5)
band11_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band11_history = band11_model.fit(band11_train_generator,
                                 steps_per_epoch = num_train_samples//batchSize,
                                 epochs = epochs,
                                 validation_data = band11_val_generator,
                                 validation_steps = num_val_samples//batchSize,
                                 verbose=1)

In [ ]:
show_final_history(band11_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='11')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred11 = band11_model.predict(test_tifs)
test_pred11 = np.argmax(test_pred11, axis=1)
test_pred11.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred11)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 11 Confusion Matrix")
plt.grid(False)
plt.show()

# Band 12 [20m]

In [ ]:
band12_train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/train.csv", band='12', batchSize=10, num_classes=10)
band12_val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSATallBands/validation.csv", band='12', batchSize=10, num_classes=10)

In [ ]:
band12_model = convModel(input_shape=(64,64,1), numClasses=10)

In [ ]:
opt = Adam(lr=1e-5)
band12_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
batchSize = 100

band12_history = band12_model.fit(band12_train_generator,
                                 steps_per_epoch = num_train_samples//batchSize,
                                 epochs = epochs,
                                 validation_data = band12_val_generator,
                                 validation_steps = num_val_samples//batchSize,
                                 verbose=1)

In [ ]:
show_final_history(band12_history)

In [ ]:
test_tifs, test_labels = obtain_tif_images(csv_file="../input/eurosat-dataset/EuroSATallBands/test.csv", band='12')

test_tifs.shape, test_labels.shape

In [ ]:
test_pred12 = band12_model.predict(test_tifs)
test_pred12 = np.argmax(test_pred12, axis=1)

test_pred12.shape

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred12)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names, title="Band 12 Confusion Matrix")
plt.grid(False)
plt.show();